In [1]:
### Module 11 Carrie Gold
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install transformers==4.39.1

In [3]:
#pip show transformers 

In [4]:
import requests
from pathlib import Path

def download_file(url, save_path):
    """Download a file from a URL and save it to the specified path."""
    response = requests.get(url)
    response.raise_for_status()  # Check if the download was successful

    with open(save_path, 'wb') as f:
        f.write(response.content)
    print(f"File downloaded successfully: {save_path}")

# URL of the dataset
url_train = "https://github.com/UUDeCART/decart_rule_based_nlp/raw/master/data/training_v2.zip"
# Path where you want to save the downloaded file
save_path_train = Path("training_v2.zip")

download_file(url_train, save_path_train)

File downloaded successfully: training_v2.zip


In [5]:
url_test = "https://github.com/UUDeCART/decart_rule_based_nlp/raw/master/data/test_v2.zip"
save_path_test = Path("test_v2.zip")

download_file(url_test, save_path_test)

File downloaded successfully: test_v2.zip


In [6]:
import zipfile

zip_path = save_path_test
extract_path = save_path_test.parent / "test_v2"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"File unzipped successfully to: {extract_path}")


File unzipped successfully to: test_v2


In [7]:
import zipfile

zip_path = "training_v2.zip"
extract_path = "training_v2"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"File unzipped successfully to: {extract_path}")

File unzipped successfully to: training_v2


In [8]:
pip install quicksectx

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install git+https://github.com/medspacy/medspacy_io

  Cloning https://github.com/medspacy/medspacy_io to c:\users\carri\appdata\local\temp\pip-req-build-rxb32wzh
  Resolved https://github.com/medspacy/medspacy_io to commit 52682d9b208cafb65733ac2aef9d6cd3f139df63
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/medspacy/medspacy_io 'C:\Users\carri\AppData\Local\Temp\pip-req-build-rxb32wzh'


In [11]:
from spacy.lang.en import English
from medspacy_io.reader import BratDocReader
from medspacy_io.reader import BratDirReader
import spacy
from pathlib import Path
from medspacy_io.vectorizer import Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [12]:
# The dataset files does not include schema configuration, let's create one
concepts=['EVIDENCE_OF_PNEUMONIA', 'PNEUMONIA_DOC_NO', 'PNEUMONIA_DOC_YES']
lines=['[entities]']+concepts
Path('annotation.conf').write_text('\n'.join(lines))


67

In [13]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 330.3 kB/s eta 0:00:39
     ---------------------------------------- 0.1/12.8 MB 1.5 MB/s eta 0:00:09
     - -------------------------------------- 0.5/12.8 MB 4.2 MB/s eta 0:00:03
     ---- ----------------------------------- 1.4/12.8 MB 9.0 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.8 MB 12.8 MB/s eta 0:00:01
     ---------- ----------------------------- 3.3/12.8 MB 13.9 MB/s eta 0:00:01
     ------------- -------------------------- 4.3/12.8 MB 16.2 MB/s eta 0:00:01
     ----------------- ---------------------- 5.5/12.8 MB 17.7 MB/s eta 0:00:01
     -------------------- ------------------- 6.7/12.8 MB 19.5 MB/s eta 0:00:01
     ------------------------- -------------- 8.0/12.8 MB 20.6 MB/s eta 0:00:01
     ---------------------------- ----------- 9.2/12.8 MB 21.8 MB/s eta 0:00:01
     ------------------------------- -------- 9.9/1

In [14]:
# set up the Brat reader
nlp=spacy.load("en_core_web_sm", disable=['ner'])
dir_reader = BratDirReader(nlp=nlp, support_overlap=True, recursive=True, schema_file='annotation.conf')

found annotation.conf file


In [15]:
# This function will read brat annotation files and convert the snippet annotation into sentence labelled dataframe
def convert2df(data_folder):
  # read brat annotation into spaCy doc object.
  docs = dir_reader.read(txt_dir=data_folder)
  # convert snippet label into sentence-level labels and generate pandas dataframe
  df = Vectorizer.docs_to_sents_df(docs, track_doc_name=True, sent_window=2)
  # remove document-level labels
  df=df[~df['y'].str.contains('_DOC_')]
  return df[['X','y']]

In [16]:
train_df=convert2df('training_v2')
test_df=convert2df('test_v2')

In [17]:
print(train_df[0:3])

                                                                                                                                                                                  X  \
1  The pulmonary vasculature is normal and there is no pneumothorax\n     on this supine radiograph.   A consolidation with air bronchograms in the left\n     lower lobe is new.     
2                                  A consolidation with air bronchograms in the left\n     lower lobe is new.   No right-sided consolidations or effusions are noted.\n     \n        
3                                                No right-sided consolidations or effusions are noted.\n     \n      IMPRESSION:  Left lower lobe process suggesting pneumonia.\n\n   

                       y  
1  EVIDENCE_OF_PNEUMONIA  
2  EVIDENCE_OF_PNEUMONIA  
3  EVIDENCE_OF_PNEUMONIA  


In [18]:
print(test_df[0:3])

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            X  \
1  [**2993-7-25**] 4:58 PM\n     CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 6033**]\n     Reason: assess for infiltrate                                       \n     ______________________________________________________________________________\n     UNDERLYING MEDICAL CONDITION:\n      35 year old man w

In [19]:
def to_attr_classify(df:pd.DataFrame):
    keywords = ['opacity', 'infiltrate', 'pneumonia', 'effusion', 'consolidation']

    # Function to find offsets of keywords in sentences
    def find_offsets(sentence, keywords):
        offsets = []
        for keyword in keywords:
            start = sentence.find(keyword)
            if start != -1:
                end = start + len(keyword)
                offsets.append((keyword, start, end))
        return offsets

    # Apply the function and expand the list of offsets into a new dataframe
    rows = []
    for i, row in train_df.iterrows():
        offsets = find_offsets(row['X'].lower(), keywords)
        for keyword, start, end in offsets:
            rows.append({'X': row['X'], 'Keyword': keyword, 'Start': start, 'End': end, 'y':row['y']})

    # Creating a new dataframe from the rows with keywords found
    filtered_df = pd.DataFrame(rows)
    filtered_df['y']=filtered_df['y'].map(lambda x: 'NotAffirmed' if x == 'NEG' else 'Affirmed')
    return filtered_df

In [20]:
train_adf=to_attr_classify(train_df)

In [21]:
### Function defined with train_df file, so changed test_df to point to the same data set 
def to_attr_classify(df:pd.DataFrame):
    keywords = ['opacity', 'infiltrate', 'pneumonia', 'effusion', 'consolidation']

    # Function to find offsets of keywords in sentences
    def find_offsets(sentence, keywords):
        offsets = []
        for keyword in keywords:
            start = sentence.find(keyword)
            if start != -1:
                end = start + len(keyword)
                offsets.append((keyword, start, end))
        return offsets

    # Apply the function and expand the list of offsets into a new dataframe
    rows = []
    for i, row in test_df.iterrows():
        offsets = find_offsets(row['X'].lower(), keywords)
        for keyword, start, end in offsets:
            rows.append({'X': row['X'], 'Keyword': keyword, 'Start': start, 'End': end, 'y':row['y']})

    # Creating a new dataframe from the rows with keywords found
    filtered_df = pd.DataFrame(rows)
    filtered_df['y']=filtered_df['y'].map(lambda x: 'NotAffirmed' if x == 'NEG' else 'Affirmed')
    return filtered_df

In [22]:
test_adf=to_attr_classify(test_df)

In [23]:
train_adf[33:40]

,X,Keyword,Start,End,y
33,"PE on VQ, with workup for probable malignancy. \n REASON FOR THIS EXAMINATION:\n Pleural effusions?",effusion,100,108,NotAffirmed
34,REASON FOR THIS EXAMINATION:\n Pleural effusions? Nodules? \n ______________________________________________________________________________\n FINAL REPORT\n HISTORY:,effusion,43,51,NotAffirmed
35,The lungs are otherwise clear.\n There are no pleural effusions.,effusion,58,66,NotAffirmed
36,There are no pleural effusions. Osseous and soft-tissue structures are\n unremarkable.\n \n,effusion,21,29,NotAffirmed
37,Cardiac and mediastinal\n contours are within normal limits for technique. There is a layering left\n pleural effusion and adjacent left basilar atelectasis.,effusion,118,126,NotAffirmed
38,There is a layering left\n pleural effusion and adjacent left basilar atelectasis. Calcified plaque is\n present along the left hemidiaphragm.,effusion,38,46,NotAffirmed
39,"IMPRESSION: AP chest compared to [**6-21**]:\n \n Bibasilar consolidation has increased substantially, particularly on the\n right, now accompanied by a small-to-moderate pleural effusion. There is some\n pulmonary edema, heart size is normal, moderate azygous distention is stable,\n and the lung findings are concerning for progressive pneumonia.",pneumonia,356,365,Affirmed


In [24]:
test_adf[33:40]

,X,Keyword,Start,End,y
33,IMPRESSION: No focal consolidations or congestive heart failure identified.\n\n,consolidation,21,34,NotAffirmed
34,"There is\n slight further resolution of the pulmonary edema. Again, note is made of left\n basilar and paracardiac atelectasis and small left pleural effusion.\n",effusion,158,166,NotAffirmed
35,"Again, note is made of left\n basilar and paracardiac atelectasis and small left pleural effusion.\n Cardiomegaly.",effusion,92,100,NotAffirmed
36,"The patient is status post median sternotomy, CABG and\n MVR. There is pulmonary vascular congestion along with a left-sided pleural\n effusion and patchy atelectasis in the left lower lobe.",effusion,143,151,NotAffirmed
37,There is pulmonary vascular congestion along with a left-sided pleural\n effusion and patchy atelectasis in the left lower lobe. The congestive heart\n failure has slightly worsened since the prior study.\n \n,effusion,76,84,NotAffirmed
38,"In the interval time from\n [**2924-4-10**] at 05:35, there has been development of subcutaneous air on the\n right. The right pleural effusion is slightly larger.",effusion,143,151,NotAffirmed
39,The right pleural effusion is slightly larger. A tiny pleural\n effusion is unchanged.,effusion,18,26,NotAffirmed


In [25]:
train_adf.value_counts("y")

y
NotAffirmed    254
Affirmed       153
Name: count, dtype: int64

In [26]:
test_adf.value_counts("y")

y
NotAffirmed    220
Affirmed       204
Name: count, dtype: int64

Assignment 1 Let's assume all the keyword identified above are true labels of NER. Now you would want to find out among these identified concepts, which are "Affirmed" statement, and which are not.

Let's see if a BERT sequence classifier can perform well on this task. You can try following the tutorial here: https://huggingface.co/docs/transformers/en/tasks/sequence_classification

In [27]:
from transformers import AutoTokenizer

In [85]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [86]:
def preprocess_function(examples):
    return tokenizer(examples["X"], truncation=True)

In [30]:
### train_adf, test_adf, X: Sentence, Keyword (effusion, opacity), Start, End, y: NotAffirmed / Affirmed

In [31]:
#pip install datasets 

In [87]:
from datasets import Dataset

In [88]:
train_adf_dataset = Dataset.from_pandas(train_adf)
test_adf_dataset = Dataset.from_pandas(test_adf)


In [89]:
### Add a column with Not/Affirmed labels
train_adf_dataset = train_adf_dataset.map(lambda example: {'labels': 1 if example['y'] == 'Affirmed' else 0})

Map:   0%|          | 0/407 [00:00<?, ? examples/s]

In [90]:
train_adf_dataset[0:3]

{'X': ['The pulmonary vasculature is normal and there is no pneumothorax\n     on this supine radiograph.   A consolidation with air bronchograms in the left\n     lower lobe is new.  ',
  'A consolidation with air bronchograms in the left\n     lower lobe is new.   No right-sided consolidations or effusions are noted.\n     \n     ',
  'A consolidation with air bronchograms in the left\n     lower lobe is new.   No right-sided consolidations or effusions are noted.\n     \n     '],
 'Keyword': ['consolidation', 'effusion', 'consolidation'],
 'Start': [101, 109, 2],
 'End': [114, 117, 15],
 'y': ['Affirmed', 'Affirmed', 'Affirmed'],
 'labels': [1, 1, 1]}

In [91]:
test_adf_dataset = test_adf_dataset.map(lambda example: {'labels': 1 if example['y'] == 'Affirmed' else 0})

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

In [92]:
test_adf_dataset[0:3]

{'X': ['[**2993-7-25**] 4:58 PM\n     CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 6033**]\n     Reason: assess for infiltrate                                       \n     ______________________________________________________________________________\n     UNDERLYING MEDICAL CONDITION:\n      35 year old man with s/p cardiac arrest                                         \n     REASON FOR THIS EXAMINATION:\n      assess for infiltrate                                                           \n     ______________________________________________________________________________\n     ',
  '[**Clip Number (Radiology) 6033**]\n     Reason: assess for infiltrate                                       \n     ______________________________________________________________________________\n     UNDERLYING MEDICAL CONDITION:\n      35 year old man with s/p cardiac arrest                                         \n     REASON FOR THIS EXAMINATION

In [96]:
test_adf_dataset

Dataset({
    features: ['X', 'Keyword', 'Start', 'End', 'y', 'labels'],
    num_rows: 424
})

In [97]:
train_adf_tok = train_adf_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/407 [00:00<?, ? examples/s]

In [98]:
test_adf_tok = test_adf_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

In [99]:
train_adf_tok 

Dataset({
    features: ['X', 'Keyword', 'Start', 'End', 'y', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 407
})

In [76]:
#train_adf_tok[1]

In [100]:
test_adf_tok 

Dataset({
    features: ['X', 'Keyword', 'Start', 'End', 'y', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 424
})

In [39]:
import tensorflow as tf

2024-03-24 14:08:52,533 tensorflow   WARNING  From c:\Users\carri\anaconda3\envs\bmi.biobert_env\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



In [109]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [113]:
### id2label = {0: "NEGATIVE", 1: "POSITIVE"}
### label2id = {"NEGATIVE": 0, "POSITIVE": 1}
id2label = {0: "NotAffirmed", 1: "Affirmed"}
label2id = {"NotAffirmed": 0, "Affirmed": 1}

In [114]:
from transformers import create_optimizer

In [115]:
from transformers import TFAutoModelForSequenceClassification

In [140]:
batch_size = 16
num_epochs = 3
batches_per_epoch = len(train_adf_tok) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [142]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [143]:
tf_train_set = model.prepare_tf_dataset(
    train_adf_tok,
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [144]:
tf_validation_set = model.prepare_tf_dataset(
    test_adf_tok,
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [147]:
model.compile(optimizer=optimizer)  

In [149]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)

Epoch 1/3
25/25 [==============================] - 130s 5s/step - loss: 0.5781 - val_loss: 0.5011
Epoch 2/3
25/25 [==============================] - 121s 5s/step - loss: 0.3551 - val_loss: 0.3574
Epoch 3/3
25/25 [==============================] - 126s 5s/step - loss: 0.2458 - val_loss: 0.3140


In [150]:
from sklearn.metrics import classification_report
y_pred = model.predict(tf_validation_set).logits.argmax(axis=-1)
y_true = [example['labels'] for example in test_adf_tok]
report = classification_report(y_true, y_pred, target_names=['NotAffirmed', 'Affirmed'])

print(report)

27/27 [==============================] - 25s 877ms/step
              precision    recall  f1-score   support

 NotAffirmed       0.93      0.85      0.89       220
    Affirmed       0.86      0.93      0.89       204

    accuracy                           0.89       424
   macro avg       0.89      0.89      0.89       424
weighted avg       0.89      0.89      0.89       424



Assignment 2
Now, you will want to identify the errors your model made. List the errors observed. What are your thoughts on the potential causes and possible solutions?

In [153]:
# Get the model predictions on the validation set
y_pred = model.predict(tf_validation_set).logits.argmax(axis=-1)
# Get the true labels from the validation set
y_true = [example['labels'] for example in test_adf_tok]
# Compare the predicted labels with the true labels and identify the errors
errors = [(x, y) for x, y in zip(y_true, y_pred) if x != y]
# Print the errors
#print(errors)
error_X = [test_adf_tok[i]['X'] for i in range(len(errors))]
print(error_X)


27/27 [==============================] - 27s 978ms/step
['[**2993-7-25**] 4:58 PM\n     CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 6033**]\n     Reason: assess for infiltrate                                       \n     ______________________________________________________________________________\n     UNDERLYING MEDICAL CONDITION:\n      35 year old man with s/p cardiac arrest                                         \n     REASON FOR THIS EXAMINATION:\n      assess for infiltrate                                                           \n     ______________________________________________________________________________\n     ', '[**Clip Number (Radiology) 6033**]\n     Reason: assess for infiltrate                                       \n     ______________________________________________________________________________\n     UNDERLYING MEDICAL CONDITION:\n      35 year old man with s/p cardiac arrest                           

Assignment 3
Now, let's try a simple yet proven effective approach. I'm not certain if it will make some difference on the "mimic" dataset, but we'll see :)
Instead of using the X directly as the input, let's try inserting the "[SEP]" token both before and after the keyword (What is "[SEP]"?). Then, use this revised string as the input to train your model.
